# Data Cleaned

In [54]:
import numpy as np
import pandas as pd

In [55]:
df_vacc = pd.read_csv('child_vaccination_data.csv')
num_rows_original = df_vacc.shape[0]
df_vacc.shape

(118423, 10)

In [56]:
df_vacc.head()

,Vaccine,Dose,Geography Type,Geography,Birth Year/Birth Cohort,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
0,Influenza,NaN,States/Local Areas,Tennessee,2014-2017,Poverty,>400% FPL,64.2,55.3 to 73.1,189.0
1,Hep A,≥1 Dose,States/Local Areas,Tennessee,2016-2019,Race and Ethnicity,Hispanic,86.4,76.8 to 95.9,187.0
2,PCV,≥4 Doses,States/Local Areas,Tennessee,2016-2019,Insurance Coverage,Private Insurance Only,89.3,85.4 to 93.2,612.0
3,Influenza,NaN,States/Local Areas,Tennessee,2014-2017,Poverty,133% to <400% FPL,51.4,44.0 to 58.8,342.0
4,Influenza,NaN,States/Local Areas,Tennessee,2014-2017,Poverty,<133% FPL,39.0,31.9 to 46.1,298.0


In [57]:
# pivot 'Dimension Type' into new columns (with 'Dimension' containing values for new columns)
df_vacc = pd.pivot(df_vacc, 
                   index=['Vaccine', 'Dose', 'Geography Type', 'Geography', 'Birth Year/Birth Cohort', \
                          'Dimension', 'Estimate (%)', '95% CI (%)', 'Sample Size'], # identifier for each row
                   columns=['Dimension Type'], # column containing new column names
                   values='Dimension')         # values for new columns

df_vacc.reset_index(inplace=True) 
df_vacc.rename_axis(None, axis=1, inplace=True) # remove the named index

In [58]:
# drop 'Dimension' column as it is no longer needed after pivot
df_vacc.drop(columns='Dimension', inplace = True)

In [59]:
# split 'Birth Year/Birth Cohort' column into two columns by looking for hyphen in values
df_vacc['Birth Year'] = np.where(df_vacc['Birth Year/Birth Cohort']\
                                 .str.contains('-'), np.nan, df_vacc['Birth Year/Birth Cohort'])
df_vacc['Birth Cohort'] = np.where(df_vacc['Birth Year/Birth Cohort']\
                                   .str.contains('-'), df_vacc['Birth Year/Birth Cohort'], np.nan)

In [60]:
# drop 'Birth Year/Birth Cohort' column as it is no longer needed after split
df_vacc.drop(columns='Birth Year/Birth Cohort', inplace = True)

In [61]:
# reorder columns
df_vacc = df_vacc[['Vaccine', 'Dose', 'Geography Type', 'Geography', 'Birth Year', 'Birth Cohort', \
                   'Estimate (%)', '95% CI (%)', 'Overall', 'Age', 'Race and Ethnicity', 'Poverty', \
                    'Insurance Coverage', 'Urbanicity', 'Sample Size']]

In [62]:
# check for NA in Estimate (%) which will be column used in all data visualizations
df_vacc['Estimate (%)'].isna().sum()

58

In [63]:
# drop rows with NA in Estimate (%) which will be column used in all data visualizations
df_vacc.dropna(subset=['Estimate (%)'], inplace=True)

num_rows_current = df_vacc.shape[0]

print(num_rows_original - num_rows_current, "rows dropped")

58 rows dropped


In [64]:
df_vacc.sample(10)

,Vaccine,Dose,Geography Type,Geography,Birth Year,Birth Cohort,Estimate (%),95% CI (%),Overall,Age,Race and Ethnicity,Poverty,Insurance Coverage,Urbanicity,Sample Size
102874,Polio,≥2 Doses,States/Local Areas,TX-Dallas County,NaN,2014-2015,80.9,75.9 to 85.2,NaN,5 Months,NaN,NaN,NaN,NaN,287.0
26751,Hep A,≥2 Doses,States/Local Areas,Georgia,NaN,2016-2019,78.5,66.5 to 90.4,NaN,NaN,Hispanic,NaN,NaN,NaN,171.0
26470,Hep A,≥2 Doses,States/Local Areas,Connecticut,NaN,2019-2020,89.6,83.7 to 94.0,NaN,35 Months,NaN,NaN,NaN,NaN,466.0
14888,DTaP,≥4 Doses,HHS Regions/National,Region 6,NaN,2019-2020,77.3,73.9 to 80.5,NaN,24 Months,NaN,NaN,NaN,NaN,3564.0
94643,PCV,≥4 Doses,States/Local Areas,IL-City of Chicago,NaN,2014-2017,81.7,75.0 to 88.5,NaN,NaN,Hispanic,NaN,NaN,NaN,170.0
23202,Hep A,≥1 Dose,States/Local Areas,New Jersey,2013,NaN,72.1,64.1 to 78.9,NaN,19 Months,NaN,NaN,NaN,NaN,252.0
80477,MMR,≥1 Dose,States/Local Areas,North Dakota,NaN,2014-2017,92.9,87.6 to 98.3,NaN,NaN,"Other or Multiple Races, Non-Hispanic",NaN,NaN,NaN,123.0
56661,Hib,Primary Series,States/Local Areas,Idaho,NaN,2015-2016,91.3,87.3 to 94.2,NaN,19 Months,NaN,NaN,NaN,NaN,404.0
83925,PCV,≥2 Doses,HHS Regions/National,Region 1,2014,NaN,85.5,82.5 to 88.1,NaN,5 Months,NaN,NaN,NaN,NaN,1564.0
2170,Combined 7 Series,NaN,States/Local Areas,Montana,2017,NaN,74.7,66.7 to 82.0,NaN,35 Months,NaN,NaN,NaN,NaN,265.0


In [65]:
# saved cleaned data to csv
df_vacc.to_csv('child_vaccination_data_cleaned.csv', index=False)